In [1]:
# 라이브러리 불러오기
import os
import json
import time
from tqdm import tqdm
from glob import glob
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [2]:
# 데이터 로딩
import os
import json
from glob import glob

dummy_dir = "/home/NEUL77/AI-Engineer/data/dummy"
jsonl_files = glob(os.path.join(dummy_dir, "*.jsonl"))

docs = []

for file_path in jsonl_files:
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line.strip())
            text = data["text"]
            metadata = data.get("metadata", {})
            docs.append({"text": text, "metadata": metadata})

In [3]:
# 출력 데이터 형식 정의
class Topic(BaseModel):
    Question: str = Field(description="지문을 기반으로 생성된 질문")
    Answer: str = Field(description="질문에 대한 답변")

parser = JsonOutputParser(pydantic_object=Topic)

# 한글 Prompt 템플릿 정의
EXAMPLES = """
예시 출력:
[
  {
    "Question": "이 시스템의 유지관리 요구사항은 무엇인가요?",
    "Answer": "시스템의 안정적 운영을 위해 장애 관리, 성능 관리, 변경 관리 등의 요구사항이 포함되어야 하며, 정기적인 모니터링이 필요합니다."
  },
  {
    "Question": "계약 이행 책임자는 어떤 자격이 요구되나요?",
    "Answer": "계약 이행 책임자는 해당 분야의 실무 경력과 자격증을 보유하고 있어야 하며, 프로젝트 전체에 대한 관리 책임을 집니다."
  }
]
"""

prompt_template = """
당신은 정부 정책 및 기술 보고서를 이해하고 질문-답변 데이터를 생성하는 전문가입니다.

아래 문서를 바탕으로 총 {num_dataset}개의 질문-답변 쌍을 생성하세요. 단, 다음 조건을 반드시 지켜야 합니다:

### 지문 이해 기준
- 문서 내용을 정확히 반영해야 하며, 사실과 다른 내용은 포함하지 마세요.
- 질문은 단답형이 아닌 설명형 또는 복합 정보 추출형으로 구성하세요.
- 답변은 문서에 있는 정보만을 바탕으로 논리적으로 정리해 작성하세요.

{examples}

### 출력 형식 (JSON 배열로 출력)
[
  {{
    "Question": "질문 내용",
    "Answer": "답변 내용"
  }},
  ...
]

문서 내용:
-----
{context}
-----

{format_instructions}
"""


# PromptTemplate 생성
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "num_dataset"],
    partial_variables={
        "examples": EXAMPLES,
        "format_instructions": parser.get_format_instructions()
    },
)


In [4]:
docs

[{'text': '제 안 요 청 서\n사 업 명 2025년 아이돌봄인력 인적성 검사 정보시스템 운영\n주관기관 한국건강가정진흥원\n2024. 12.\n전화번호\n성 명 소 속 e-mail\n(팩스번호)\n담 당\n가족친화사업본부 02-3479-7701\n진용수 ysjin@kihf.or.kr\n돌봄지원부 02-3479-7717\n\n[Table]\n|사 업 명|2025년 아이돌봄인력 인적성 검사 정보시스템 운영|\n|---|---|\n|주관기관|한국건강가정진흥원|\n\n|담 당|성 명|소 속|전화번호\n(팩스번호)|e-mail|\n|---|---|---|---|---|\n||진용수|가족친화사업본부\n돌봄지원부|02-3479-7701\n02-3479-7717|ysjin@kihf.or.kr|',
  'metadata': {'source_id': '한국건강가정진흥원_2025년_아이돌봄인력_인적성_검사_정보시스템_운영',
   'chunk_id': 'chunk-0001'}},
 {'text': '- 목 차 -\nⅠ. 사업 개요\n1. 추진배경··························· 1\n2. 사업개요··························· 1\n3. 사업범위··························· 2\n4. 기대효과··························· 2\nⅡ. 사업 현황\n1. 업무현황··························· 3\n2. 정보화현황·························· 3\nⅢ. 사업 추진방안\n1. 추진목표··························· 5\n2. 추진체계··························· 5\n3. 추진일정··························· 6\n4. 추진방안··························· 6\nⅣ. 제안요청 내용\n1. 목표시스템 구성도·······················7\n2. 사업내용··················

In [5]:
docs[100]

{'text': '(설계보상 대상자 선정공문 등) 제출 필요(증빙서류 미제출시 서류 보완요청\n없이 실적 불인정)\n[금번 입찰시 실적 산정방법]\n1. 금번 입찰에 공동도급으로 참여하는 경우, 평가대상항목 건수/금액/해외 결과\n(점수)를 아래 수식에 따라 합산\n- 유사사업 수행실적 점수 = ITS 현장시스템 구축 계획 점수(70%) + ICT 시스템\n구축 계획 점수(30%)\n* 상기의 수식 및 비율은 공동도급사 간 실제 참여지분율 구성과는 관련 없음\n- 동일 분야에 복수의 기업이 공동으로 참여 또는 참여기업이 복수의 분야에\n수행실적을 제출하는 경우, 참여지분율(동일업종 참여지분율 합을 100%로 합산)을\n고려하여 실적 산정(공동수급 약정서에 참여사별 담당 업무범위 명시 필수)\n- 15 -',
 'metadata': {'source_id': '한국수출입은행__긴급_모잠비크_마푸토_지능형교통시스템_ITS_구축사업',
  'chunk_id': 'chunk-0017'}}

In [6]:
len(docs)

9555

In [9]:
import random
from collections import defaultdict

# QA 최대 생성 수
MAX_QA_PER_SOURCE = 3

# source_id별 문서 모으기
source_id_to_docs = defaultdict(list)
for doc in docs:
    source_id = doc["metadata"].get("source_id", "")
    if source_id:
        source_id_to_docs[source_id].append(doc)

# 모델 및 체인 정의
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm | parser

# 저장 경로 
output_path = "/home/NEUL77/AI-Engineer/data/train/qa_dataset_V2.jsonl"

with open(output_path, "a", encoding="utf-8") as f:
    start_time = time.time()
    total_qa = 0

    for i, (source_id, doc_list) in enumerate(tqdm(source_id_to_docs.items(), desc="Generating QA")):
        # 랜덤하게 최대 3개 선택
        sampled_docs = random.sample(doc_list, min(MAX_QA_PER_SOURCE, len(doc_list)))

        for doc in sampled_docs:
            context_text = doc["text"]
            chunk_id = doc["metadata"].get("chunk_id", "")

            print(f"[{total_qa+1}] Generating QA for: {source_id} - {chunk_id}")
            try:
                items = chain.invoke({'num_dataset': 1, 'context': context_text})
                for item in items:
                    item["source_id"] = source_id
                    item["chunk_id"] = chunk_id
                    json_line = json.dumps(item, ensure_ascii=False)
                    f.write(json_line + "\n")
                    total_qa += 1
            except Exception as e:
                print(f"[{source_id}] Error: {e}")

        # 진행 상황 출력
        if i % 10 == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i + 1)
            remaining = len(source_id_to_docs) - (i + 1)
            eta = avg_time * remaining
            print(f"남은 예상 시간: {eta / 60:.2f}분")

print(f"[✔] 총 생성된 QA 수: {total_qa}")
print(f"[✔] QA 데이터셋 저장 완료: {output_path}")

Generating QA:   0%|          | 0/100 [00:00<?, ?it/s]

[1] Generating QA for: 한국건강가정진흥원_2025년_아이돌봄인력_인적성_검사_정보시스템_운영 - chunk-0052
[2] Generating QA for: 한국건강가정진흥원_2025년_아이돌봄인력_인적성_검사_정보시스템_운영 - chunk-0008
[3] Generating QA for: 한국건강가정진흥원_2025년_아이돌봄인력_인적성_검사_정보시스템_운영 - chunk-0025


Generating QA:   1%|          | 1/100 [00:13<22:57, 13.92s/it]

남은 예상 시간: 22.97분
[4] Generating QA for: 한국수출입은행__긴급_모잠비크_마푸토_지능형교통시스템_ITS_구축사업 - chunk-0031
[5] Generating QA for: 한국수출입은행__긴급_모잠비크_마푸토_지능형교통시스템_ITS_구축사업 - chunk-0080
[6] Generating QA for: 한국수출입은행__긴급_모잠비크_마푸토_지능형교통시스템_ITS_구축사업 - chunk-0052


Generating QA:   2%|▏         | 2/100 [00:28<23:21, 14.30s/it]

...

[295] Generating QA for: 한국철도공사_용역__예약발매시스템_개량_ISMP_용역 - chunk-0092
[296] Generating QA for: 한국철도공사_용역__예약발매시스템_개량_ISMP_용역 - chunk-0038
[297] Generating QA for: 한국철도공사_용역__예약발매시스템_개량_ISMP_용역 - chunk-0009


Generating QA:  99%|█████████▉| 99/100 [20:10<00:10, 10.60s/it]

[298] Generating QA for: 한국농수산식품유통공사_농산물가격안정기금_정부예산회계연계시스템_ - chunk-0075
[299] Generating QA for: 한국농수산식품유통공사_농산물가격안정기금_정부예산회계연계시스템_ - chunk-0060
[300] Generating QA for: 한국농수산식품유통공사_농산물가격안정기금_정부예산회계연계시스템_ - chunk-0014


Generating QA: 100%|██████████| 100/100 [20:23<00:00, 12.23s/it]

[✔] 총 생성된 QA 수: 300
[✔] QA 데이터셋 저장 완료: /home/NEUL77/AI-Engineer/data/train/qa_dataset_V2.jsonl


In [ ]:
def build_context_dict(docs):
    context_dict = {}
    for doc in docs:
        source_id = doc["metadata"].get("source_id")
        chunk_id = doc["metadata"].get("chunk_id")
        if source_id and chunk_id:
            context_dict[(source_id, chunk_id)] = doc["text"]
    return context_dict

In [ ]:
def convert_to_sft_format(qa_path, context_dict, output_path):
    with open(qa_path, "r", encoding="utf-8") as f_in, \
         open(output_path, "w", encoding="utf-8") as f_out:
        for line in f_in:
            qa = json.loads(line.strip())
            key = (qa["source_id"], qa["chunk_id"])
            context = context_dict.get(key, "")
            sft_item = {
                "input": f"문서 내용:\n{context}\n\n질문:\n{qa['Question']}",
                "output": qa["Answer"]
            }
            f_out.write(json.dumps(sft_item, ensure_ascii=False) + "\n")

In [5]:
# 원본 문서 불러오기
dummy_dir = "/home/NEUL77/AI-Engineer/data/dummy"
jsonl_files = glob(os.path.join(dummy_dir, "*.jsonl"))

docs = []
for file_path in jsonl_files:
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line.strip())
            docs.append({
                "text": data["text"],
                "metadata": data.get("metadata", {})
            })

# source_id + chunk_id 기반 context dict 생성
context_dict = build_context_dict(docs)

# QA + context → SFT 포맷 변환
qa_path = "/home/NEUL77/AI-Engineer/data/train/qa_dataset.jsonl"
output_path = "/home/NEUL77/AI-Engineer/data/train/qa_sft_dataset.jsonl"
convert_to_sft_format(qa_path, context_dict, output_path)
